In [1]:
import numpy as np
import pandas as pd
from math import pi, sqrt

import astropy.table #import tables
from astropy import units as u
from astropy import constants as const

from preprocessing.calc_stellar_params import calc_luminosity, calc_temp
from preprocessing.analyse_errs import classify_err
#from preprocessing.calc_sephi import get_sephi_RM17


In [2]:
# Exoplanet directory:
#NASAEA_dir = "~/Scarlett/OneDrive - Liverpool John Moores University/SEPHI_data/NASA_EA_2022_02_09.csv"
NASAEA_dir = "~/OneDrive/SEPHI_data/NASA_EA_2022_02_09.csv"
header_length = 116

# CKS directories:
CKS_stars_dir = "~/OneDrive/SEPHI_data/CKSII_2017_stars.tsv"
CKS_col_start = 100 # the row where containing the colum headers
# The length of the header in the exoplanets csv file:


In [3]:
# Read data
exoplanets = pd.read_csv(NASAEA_dir, skiprows=header_length, 
                         usecols=["pl_name", "hostname", "gaia_id", "sy_snum", "sy_pnum", "discoverymethod", "pl_orbper", "pl_orbsmax", "pl_rade", "pl_bmasse", "pl_dens", "pl_orbeccen", "pl_eqt", "pl_orbincl", "st_teff", "st_tefferr1", "st_tefferr2", "st_rad", "st_raderr1", "st_raderr2", "st_mass", "st_met", "st_lum", "st_lumerr1", "st_lumerr2", "st_logg", "st_age", "st_ageerr1", "st_ageerr2", "sy_dist", "sy_plx", "sy_gaiamag"])
# pl_orbper = orbital period [days]
# pl_orbsmax = orbit semi-major axis [au]
#exculuded "st_spectype" due to csv formatting

In [4]:
#print(exoplanets.head())

In [5]:
# Get rid of exoplanets whose star's haven't been observed by Gaia
exoplanets.dropna(subset=["gaia_id"], inplace=True)

In [6]:
# Not many of the exoplanets have the stellar age listed. Working out how many have stellar age:
#print("Number of exoplanets with no stellar age listed: ", exoplanets["st_age"].isna().sum())
#print("Number of exoplanets with stellar age listed: ", exoplanets["st_age"].notna().sum())

In [7]:
# A few don't have stellar effective temperature listed (st_teff):
#print("Number of exoplanets with no teff listed: ", exoplanets["st_teff"].isna().sum())
#print("Number of exoplanets with teff listed: ", exoplanets["st_teff"].notna().sum())

In [8]:
# Some don't have stellar radius listed
#print("Number of exoplanets with no st_rad listed: ", exoplanets["st_rad"].isna().sum())
#print("Number of exoplanets with st_rad listed: ", exoplanets["st_rad"].notna().sum())

#print("Number of exoplanets with st_raderr1 listed: ", exoplanets["st_raderr1"].notna().sum())
#print("Number of exoplanets with st_raderr2 listed: ", exoplanets["st_raderr2"].notna().sum())

In [9]:
# Some don't have stellar age listed
#print("Number of exoplanets with no st_age listed: ", exoplanets["st_age"].isna().sum())
#print("Number of exoplanets with st_age listed: ", exoplanets["st_age"].notna().sum())

In [10]:
# Number of stars with st_rad and st_teff:
"""
count = 0
for i in range(exoplanets.shape[0]):
    if pd.notna(exoplanets["st_teff"].iloc[i]) and pd.notna(exoplanets["st_rad"].iloc[i]):
        count+=1
    else:
        continue
print("Number of exoplanets with st_eff and st_rad listed (can calc L): ", count)
    
# Number of stars with st_rad, st_teff and st_age:
count = 0
for i in range(exoplanets.shape[0]):
    if pd.notna(exoplanets["st_teff"].iloc[i]) and pd.notna(exoplanets["st_rad"].iloc[i]) and pd.notna(exoplanets["st_age"].iloc[i]):
        count+=1
    else:
        continue

print("Number of exoplanets with st_teff, st_rad and st_age listed (can calc SEPHI): ", count)
"""

'\ncount = 0\nfor i in range(exoplanets.shape[0]):\n    if pd.notna(exoplanets["st_teff"].iloc[i]) and pd.notna(exoplanets["st_rad"].iloc[i]):\n        count+=1\n    else:\n        continue\nprint("Number of exoplanets with st_eff and st_rad listed (can calc L): ", count)\n    \n# Number of stars with st_rad, st_teff and st_age:\ncount = 0\nfor i in range(exoplanets.shape[0]):\n    if pd.notna(exoplanets["st_teff"].iloc[i]) and pd.notna(exoplanets["st_rad"].iloc[i]) and pd.notna(exoplanets["st_age"].iloc[i]):\n        count+=1\n    else:\n        continue\n\nprint("Number of exoplanets with st_teff, st_rad and st_age listed (can calc SEPHI): ", count)\n'

In [11]:
# Not many have stellar luminosity listed (but this can be calculated):
#print("Number of exoplanets with no L listed: ", exoplanets["st_lum"].isna().sum())
#print("Number of exoplanets with L listed: ", exoplanets["st_lum"].notna().sum())

# Do more have their luminosities listed in Gaia?

In [12]:
print(exoplanets.shape[0])

4734


In [13]:
# Estimate L for stars in 'exoplanets'
luminosities = np.zeros( (exoplanets.shape[0],5) )
for i in range(exoplanets.shape[0]):
    luminosities[i] = calc_luminosity(exoplanets["st_teff"].iloc[i], exoplanets["st_tefferr1"].iloc[i], exoplanets["st_tefferr2"].iloc[i], exoplanets["st_rad"].iloc[i], exoplanets["st_raderr1"].iloc[i], exoplanets["st_raderr2"].iloc[i]) 

#print(luminosities)

In [14]:
# Add the luminosities columns to the exoplanets data frame:
exoplanets[ ["calc_L", "calc_Lerr1", "calc_L%err1", "calc_Lerr2", "calc_L%err2"] ] = luminosities
#print(exoplanets)

In [15]:
# Array to store classification:
classes_L = np.zeros( (exoplanets.shape[0]), dtype=int )
#print(classes_L[0:40])

# Classify the luminosity uncertainties using the function:
for i in range(exoplanets.shape[0]):
    classes_L[i] = classify_err(exoplanets["st_lum"].iloc[i], exoplanets["st_lumerr1"].iloc[i], exoplanets["st_lumerr2"].iloc[i], exoplanets["calc_L%err1"].iloc[i], exoplanets["calc_L%err2"].iloc[i])
                                                                                                             
#print(classes_L[0:100])
#print(len(classes_L))

In [16]:
#print("Number of exoplanets with errors in st_lum < calc_L or no dcalc_L: ", np.count_nonzero(classes_L == 1),"\nNB: there are 909 planets with st_lum listed.") 
#print("Number of exoplanets with errors in calc_L < st_lum or no dst_lum: ", np.count_nonzero(classes_L == 2))

In [17]:
# Add the crror class column to the exoplanets data frame
exoplanets[ "Lerr_class" ] = classes_L
#print(exoplanets)

In [18]:
# Estimate T for stars in 'exoplanets'
temps = np.zeros( (exoplanets.shape[0], 5) )
for i in range(exoplanets.shape[0]):
    temps[i] = calc_temp(exoplanets["st_rad"].iloc[i], exoplanets["st_raderr1"].iloc[i], exoplanets["st_raderr2"].iloc[i], exoplanets["st_lum"].iloc[i], exoplanets["st_lumerr1"].iloc[i], exoplanets["st_lumerr2"].iloc[i]) 

#print(temps)

In [19]:
# Add calc_teff to exoplanets df:
exoplanets[ ["calc_T", "calc_Terr1", "calc_T%err1", "calc_Terr2", "calc_T%err2"] ] = temps
#print(exoplanets.head(10))
# Temps and errors look good!

In [20]:
# Compare the uncertainties in st_teff and calc_teff: 

# Array to store classification:
classes_T = np.zeros( (exoplanets.shape[0]), dtype=int )

for i in range(exoplanets.shape[0]):
    classes_T[i] = classify_err(exoplanets["st_teff"].iloc[i], exoplanets["st_tefferr1"].iloc[i], exoplanets["st_tefferr2"].iloc[i], exoplanets["calc_T%err1"].iloc[i], exoplanets["calc_T%err2"].iloc[i])
                                                                                                             
#print(classes_T[0:100])
#print(len(classes_T))

In [21]:
# Counting how many st_teff have smaller error and how many temp have smaller error
#print("Number of exoplanets with errors in st_teff < calc_T or incomplete errors on calc_teff: ", np.count_nonzero(classes_T == 1),"\nNB: there are 4507 planets with st_teff listed.") 
#print("Number of exoplanets with errors in calc_T < st_teff or incomplete errors on st_teff: ", np.count_nonzero(classes_T == 2))

In [22]:
# Adding the classes_T array to the exoplanets df
exoplanets["Terr_class"] = classes_T
#print(exoplanets)

In [23]:
pl_mass = exoplanets["pl_bmasse"].to_numpy() * u.earthMass
pl_rad = exoplanets["pl_rade"].to_numpy() * u.earthRad
pl_a = exoplanets["pl_orbsmax"].to_numpy() * u.AU
#teff = exoplanets["st_teff"].to_numpy() * u.K
#lum = exoplanets["st_lum"].to_numpy() * u.dex(u.L_sun)
st_mass = exoplanets["st_mass"].to_numpy() * u.solMass
age = exoplanets["st_age"].to_numpy() * u.Gyr
print("planet mass: ", pl_mass, "\n data shape: ", pl_mass.shape)
      #, pl_rad, pl_ a)

planet mass:  [6165.6     4684.8142  1525.5     ... 1313.22     162.09249  890.     ] earthMass 
 data shape:  (4734,)


In [24]:
from preprocessing.calc_sephi import get_sephi_RM17

In [25]:
#sephi = get_sephi_RM17(pl_mass, pl_rad, pl_a, teff, lum, st_mass, age, verbose=False)

In [26]:
#print(sephi)
#print(sephi.shape)

# Checking whether uncertainties are the right sign (+/-ve)
#print( np.where( exoplanets["calc_Terr1"] <0 ) )
#print( np.where( exoplanets["calc_Terr2"] >0 ) )
#print( (np.where( np.isnan( exoplanets["calc_Terr2"].head(100)) ) ) )

In [27]:
# Collating the 'best' values for stellar luminosity

# The conditions that determine which L value is the 'best':
c1= np.where( (exoplanets["Lerr_class"].to_numpy() == 0) | (exoplanets["Lerr_class"].to_numpy() == 1) ) # indicies for which st_lum has smaller uncertainties
#print(c1[0])
#print(c1[0].size)
c2 = np.where( (exoplanets["Lerr_class"].to_numpy() == 2 ) ) # indicies for which calc_T has smaller uncertainties
c3 = np.where( (exoplanets["Lerr_class"].to_numpy() == 3 ) ) # indioies where neither st_lum nor calc_T have uncertainties, or both st_lum and calc_T are NaN

# Empty array to store the 'best' luminosities:
lum = np.zeros(exoplanets.shape[0])
print(lum.size)

lum[c1] = exoplanets["st_lum"].iloc[c1] # st_lum has smaller uncertainties
lum[c2] = exoplanets["calc_L"].iloc[c2] # calc_L has smaller uncertainties
lum[c3] = exoplanets["calc_L"].iloc[c3] # these are all NaN

lum = lum * u.dex(u.L_sun)
print(lum)

# NB: lums and teffs that aren't NaN but class 3 do not have uncertainties. Are these reliable results?

4734
[2.243      2.40120052 1.7671425  ...        nan 1.8        1.83076776] dex(solLum)


In [28]:
# Collating the 'best' values for effective temperature

# The conditions that determine which teff value is the 'best':
c1= np.where( (exoplanets["Terr_class"].to_numpy() == 0) | (exoplanets["Terr_class"].to_numpy() == 1) )
#print(c1[0])
#print(c1[0].size)
c2 = np.where( (exoplanets["Terr_class"].to_numpy() == 2 ) )
c3 = np.where( (exoplanets["Terr_class"].to_numpy() == 3 ) )

# Empty array to store the 'best' luminosities:
teff = np.zeros(exoplanets.shape[0])
print(teff.size)

teff[c1] = exoplanets["st_teff"].iloc[c1] # st_teff has smaller uncertainties
teff[c2] = exoplanets["calc_T"].iloc[c2] # calc_T has smaller uncertainties
teff[c3] = exoplanets["calc_T"].iloc[c3] # these are all NaN

teff = teff * u.K
print(teff)

4734
[4742.         4213.         4813.         ...           nan 4856.57558202
 4780.        ] K


In [29]:
sephi = get_sephi_RM17(pl_mass, pl_rad, pl_a, teff, lum, st_mass, age, verbose=False)

Number of planets for which SEPHI cannot be calculated:  4242
Number of planets for which SEPHI can be calculated:  492


In [30]:
print(sephi)
# TODO: see how many planets have SEPHI = 0 so far

[nan nan nan ... nan nan nan]


In [37]:
exoplanets.rename(columns={ "gaia_id": "designation" }, inplace=True )

In [31]:
# TODO: match the dr2 stellar params with NASA EA

# Things form Michal's exoplanets_gaia_crossmatch:
exoplanets2 = exoplanets
print(exoplanets2["designation"])
# The Gaia ID in exoplanets is the DR followed by the star source ID (the designation)

0       Gaia DR2 3946945413106333696
1       Gaia DR2 1696798367260229376
2       Gaia DR2 1920113512486282240
3       Gaia DR2 1385293808145621504
4       Gaia DR2 2135550755683407232
                    ...             
4909     Gaia DR2 348020448377061376
4910     Gaia DR2 348020448377061376
4911     Gaia DR2 348020448377061376
4912    Gaia DR2 3794167001116433152
4913    Gaia DR2 4298361114750843904
Name: gaia_id, Length: 4734, dtype: object


In [32]:
exoplanets2["source_id"] = exoplanets2["designation"].str.rsplit(" ", n=1, expand=True)[1].astype("int64")
# This is the gaia_id without the DR listed
print(exoplanets2["source_id"])
# The same source_id as listed in Gaia

0       3946945413106333696
1       1696798367260229376
2       1920113512486282240
3       1385293808145621504
4       2135550755683407232
               ...         
4909     348020448377061376
4910     348020448377061376
4911     348020448377061376
4912    3794167001116433152
4913    4298361114750843904
Name: source_id, Length: 4734, dtype: int64


In [33]:
print(exoplanets2["hostname"])

0         11 Com
1         11 UMi
2         14 And
3         14 Her
4       16 Cyg B
          ...   
4909     ups And
4910     ups And
4911     ups And
4912     ups Leo
4913      xi Aql
Name: hostname, Length: 4734, dtype: object


In [34]:
exoplanets2.drop(["designation"], axis=1, inplace=True)
exoplanets2["Host"] = exoplanets2["hostname"].str.replace(" ", "")
print(exoplanets2["Host"])
#exoplanets2.drop_duplicates(subset=["Host"], inplace=True)
# TODO: I don't know what to do about that line, I think I want duplicates (at least later on so that all planets are listed with their respective star, even if stars are repeated)

0        11Com
1        11UMi
2        14And
3        14Her
4       16CygB
         ...  
4909    upsAnd
4910    upsAnd
4911    upsAnd
4912    upsLeo
4913     xiAql
Name: Host, Length: 4734, dtype: object


In [38]:
dr2_directory = "~/OneDrive/SEPHI_data/dr2-result.csv"

dr2 = pd.read_csv(dr2_directory, usecols=["designation", "source_id", "teff_val", "teff_percentile_lower", "teff_percentile_upper", "radius_val", "radius_percentile_lower", "radius_percentile_upper", 
                                          "lum_val", "lum_percentile_lower", "lum_percentile_upper"])


AttributeError: 'DataFrame' object has no attribute 'length'

In [39]:
print(dr2.shape[0])

7183262


In [ ]:
exoplanets2 = pd.merge(exoplanets, dr2, on="source_id")

In [35]:
print(exoplanets2)
# TODO: I lose a lot of exoplanets after merging with dr2
# This could be becasue not all the exoplanet hosts are in dr2
# Some might only be in edr3 (in which case, we defo don't have gaia L for them) 
# I don't want any removed

          pl_name hostname  sy_snum  sy_pnum  discoverymethod     pl_orbper  \
0        11 Com b   11 Com        2        1  Radial Velocity    326.030000   
1        11 UMi b   11 UMi        1        1  Radial Velocity    516.219970   
2        14 And b   14 And        1        1  Radial Velocity    185.840000   
3        14 Her b   14 Her        1        2  Radial Velocity   1773.400020   
4     HD 145675 c   14 Her        1        2  Radial Velocity  25000.000000   
...           ...      ...      ...      ...              ...           ...   
1839    ups And b  ups And        2        3  Radial Velocity      4.617033   
1840    ups And c  ups And        2        3  Radial Velocity    241.258000   
1841    ups And d  ups And        2        3  Radial Velocity   1276.460000   
1842    ups Leo b  ups Leo        1        1  Radial Velocity    385.200000   
1843     xi Aql b   xi Aql        1        1  Radial Velocity    136.750000   

      pl_orbsmax  pl_rade   pl_bmasse  pl_dens  ...

In [36]:
# TODO: Michal then does some stuff that I don't understand...
exoplanets.drop(["pl_name", "hostname"], axis=1, inplace=True)

gaia = gaia[~gaia["source_id"].isin(exoplanets["source_id"])]

gaia = gaia[4.5 < gaia["parallax"] / gaia["parallax_error"]] #removes stars with with parallax/parallax error > 4.5

# Concatenate exoplanet hosts back, however at the top of the dataframe. This way for testing purposes we later
# iterate only over first 1065 entries that are exoplanet hosts.
gaia = pd.concat([exoplanets, gaia]) #adding the exoplanet list back into the gaia df, at the top

# Calculate distance in pc and drop any stars with negative or null distance
gaia["distance_pc"] = (1. / gaia["Fparallax"]) * 1000 #closely aligned sources are only occasionally resolved in Gaia, confusion in observation-to-source matching can lead to spurious parallax values which are either very large or have a negative value very far away from zero
gaia = gaia[gaia["distance_pc"] > 0] #returns all of gaia for which distance_pc > 0 and overwrites the gaia df with it. Gets rid of all entries where distance_pc <= 0. For these entries, the solution returned by gaia is unphysical so we want to ditch it
    
# Convert from degrees to pc
gaia["ra"] = (gaia["ra"] * np.pi) / 180.
gaia["dec"] = (gaia["dec"] * np.pi) / 180.

NameError: name 'gaia' is not defined

In [ ]:
# TODO: match with CKS stellar catalogue
# TODO: redo best param search (flag where CKS used)
# TODO: calc sephi for CKS planets only
# TODO: match with CKS planet params
# TODO: add the sephi to the data frame

In [ ]:
cks_stars = pandas.read_csv(CKS_stars_dir, sep = ';', header = CKS_col_start, usecols=[ "Name", "Teff", "eTeff", "E_Teff", "Mass", "e_Mass", "e_Mass", "logA", "e_logA", "E_logA" ])
#"Npl", "logg", "e_logg", "E_logg", "[Fe/H]",
# Name = Star name: mostly Kepler Object Identifier (KOI), or Kepler Input Calalog (KIC) number
# logA (log_10(age)) [ucd=time.age] TODO: what does that last bit mean?

# TODO: match KOI with some other identifier in NASA EA or is KOI available in NASA EA?